In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
data_df=pd.read_excel('深圳市环境数据.xls',sheet_name='data')
name_df=pd.read_excel('深圳市环境数据.xls',sheet_name='name')

In [5]:
print(data_df)

        2000       2001       2002       2003       2004       2005  \
0      0.059      0.063      0.061      0.070      0.076      0.064   
1      0.027      0.027      0.018      0.020      0.024      0.021   
2      0.055      0.058      0.050      0.057      0.072      0.039   
3  38427.000  39264.000  40780.000  40872.000  43633.000  43453.000   
4   3352.000   3477.000   4287.000   5052.000   6241.000   6468.000   
5     43.080     43.800     42.240     58.630     83.370     85.390   

        2006       2007       2008       2009       2010      2011      2012  \
0      0.064      0.064      0.063      0.057      0.057     0.057     0.054   
1      0.030      0.023      0.016      0.013      0.011     0.011     0.010   
2      0.053      0.054      0.047      0.042      0.045     0.048     0.040   
3  42380.000  37957.000  33850.000  31947.000  32642.000      9482  9847.000   
4   4028.650   3261.960   4044.690   3506.620   1670.230  1 254.50   827.760   
5    107.350    161.82